## Load data

In [1]:
import config

from navernews_10days import get_comments_paths
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from soynlp.utils import DoublespaceLineCorpus

path = get_comments_paths(date='2016-10-20')
corpus = DoublespaceLineCorpus(path, iter_sent=True)

soynlp=0.0.491
added lovit_textmining_dataset


## Train Word Extractor and Tokenizer

명사 추출기를 이용하여 명사 점수를 단어 점수로 이용합니다. 뉴스 기사는 띄어쓰기가 잘 되어 있기 때문에 L-Tokenizer 를 이용하였습니다.

In [2]:
noun_extractor = LRNounExtractor_v2()
noun_scores = noun_extractor.train_extract(corpus)

word_scores = {noun:score.score for noun, score in noun_scores.items()}
tokenizer = LTokenizer(scores = word_scores)

tokenizer.tokenize('뉴스 기사를 이용하여 학습한 모델입니다', tolerance=0.3)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=1260, neg=1173, common=12
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 687560 from 228630 sents. mem=0.181 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=2775027, mem=1.653 Gb
[Noun Extractor] batch prediction was completed for 244896 words
[Noun Extractor] checked compounds. discovered 183564 compounds
[Noun Extractor] postprocessing detaching_features : 211088 -> 159349
[Noun Extractor] postprocessing ignore_features : 159349 -> 159153
[Noun Extractor] postprocessing ignore_NJ : 159153 -> 157566
[Noun Extractor] 157566 nouns (183564 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.006 Gb                    
[Noun Extractor] 74.53 % eojeols are covered


['뉴스', '기사', '를', '이용', '하여', '학습', '한', '모델', '입니다']

## Build co-occurrence matrix

L parts 에서 명사 점수의 최대값과 0.3 점 차이가 나지 않는다면 그 중 가장 긴 subword 를 단어로 추출하도록 tolerance 를 이용하였습니다. min_tf 를 10 으로 설정하여 10 번 이하로 등장한 단어에 대해서는 co-occurrence 를 계산하지 않습니다.

In [3]:
from soynlp.vectorizer import sent_to_word_contexts_matrix

def custom_tokenizer(sent):
    return tokenizer.tokenize(sent, tolerance=0.3)

x, idx2vocab = sent_to_word_contexts_matrix(
    corpus,
    windows = 2,
    min_tf = 10,
    tokenizer = custom_tokenizer,
    verbose = True
)

Create (word, contexts) matrix
  - counting word frequency from 228629 sents, mem=2.157 Gb
  - scanning (word, context) pairs from 228629 sents, mem=2.157 Gb
  - (word, context) matrix was constructed. shape = (30810, 30810)                    
  - done


학습 후 30,810 개의 단어가 학습되었습니다.

In [4]:
x.shape

(30810, 30810)

## similar words using context vector

idx2vocab 을 vocab2idx 로 만듭니다. 

In [5]:
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}

print(vocab2idx['이화여대'])
print(vocab2idx['아이오아이'])

1302
1889


sklearn.metrics.pairwise_distances 를 이용하여 '이화여대'와 context vector 가 비슷한 다른 단어를 찾습니다. 

numpy.ndarray 형식인 distance matrix (dist) 에 대하여 argsort() 를 하면, 거리 순서대로 정렬됩니다. sort() 를 하면 값이 정렬되며, argsort() 를 하면 각 값의 index 가 return 됩니다. 

여러 번 쓸 수 있도록 함수로도 만들어둡니다. 

In [6]:
import numpy as np
from sklearn.metrics import pairwise_distances

def most_similar_words(query, wv, topk=10):
    """
    query : str
    wv : word representation
    topk : num of similar terms
    """
    
    if not (query in vocab2idx):
        return []

    query_idx = vocab2idx[query]
    query_vec = wv[query_idx,:].reshape(1,-1)
    dist = pairwise_distances(query_vec, wv, metric='cosine')[0]
    similars = []

    # sorting
    for similar_idx in dist.argsort():
        
        # filtering query term
        if similar_idx == query_idx:
            continue

        if len(similars) >= topk:
            break

        # decoding index to word
        similar_word = idx2vocab[similar_idx]
        similars.append((similar_word, 1 - dist[similar_idx]))

    return similars

Cooccurrence matrix 만을 이용해도 문맥이 **매우** 확실한 단어 `아이오아이`는 다른 아이돌 이름이나 `아오아` 같은 약어가 유사어로 검색됩니다.

In [7]:
most_similar_words('아이오아이', x)

[('블랙핑크', 0.6647859272131573),
 ('트와이스', 0.6262975246594952),
 ('엑소', 0.5901119271712834),
 ('여자친구', 0.5623464164823095),
 ('노래', 0.5616676134078805),
 ('우리나라', 0.5489489482770429),
 ('에이핑크', 0.5366051901750828),
 ('아오아', 0.5336581153448168),
 ('우리', 0.5334947525137911),
 ('가수', 0.5308252449873944)]

그러나 단어 `아프리카`의 유사어는 잘 검색되지 않습니다. Co-occurrence frequency 를 representation 으로 이용하면 어느 문맥에나 등장하는 단어들에 영향을 받기 때문입니다.

In [8]:
most_similar_words('아프리카', x)

[('회사', 0.815166536698036),
 ('우리나라', 0.7957352100985269),
 ('머리', 0.7956636660982553),
 ('학교', 0.7854806600688846),
 ('우리', 0.7806264137345261),
 ('정부', 0.772928310373537),
 ('사회', 0.7697541171327323),
 ('이나라', 0.7691019000466983),
 ('스크린도어', 0.7584292635755353),
 ('뉴스', 0.756708891308895)]

## PPMI 를 이용한 term weighting

soynlp 의 pmi 를 이용하여 co-occurrence matrix 에 PMI 를 적용합니다. `min_pmi` 를 0 으로 설정하여 Positive PMI 로 만듭니다.

In [9]:
from soynlp.word import pmi

x_pmi, px, py = pmi(x, min_pmi=0.0)

PPMI 가 어느 문맥에나 등장하는 단어의 weight 를 0 으로 만들기 때문에 단어의 문맥이 뚜렷해집니다. `아이오아이`의 유사어는 크게 달라지지 않았습니다.

In [10]:
most_similar_words('아이오아이', x_pmi)

[('블랙핑크', 0.17036069200581827),
 ('트와이스', 0.16415835873231055),
 ('ioi', 0.16198705263875812),
 ('너무너무너무', 0.1551582738086552),
 ('아오아', 0.14416957968213384),
 ('블핑', 0.13958860792378958),
 ('IOI', 0.13534074388406614),
 ('컴백', 0.12791868758168912),
 ('CLC', 0.12636026506255216),
 ('신곡', 0.12568333275806576)]

그러나 `아프리카`의 유사어는 크게 바뀌었습니다. 다른 나라의 이름도 검색되며, `아프리카tv` 관련 단어인 `bj` 나 `유튜브`가 검색됩니다. 이날 대륙 `아프리카`에 관련된 기사와 `아프리카tv` 관련 기사가 모두 있었기 때문에 두 문맥이 모두 반영되어 있습니다.

그러나 representation vector 의 차원은 3 만 차원보다 큽니다.

In [11]:
most_similar_words('아프리카', x_pmi)

[('bj', 0.10852224057916593),
 ('아프리카tv', 0.07907760372410033),
 ('유튜브', 0.07811900708395747),
 ('비제이', 0.07292670984989313),
 ('수수료', 0.06970795714059574),
 ('유투브', 0.06888458719259427),
 ('플랫폼', 0.0678840033629654),
 ('난민', 0.06690176683704696),
 ('중동', 0.06468149614532082),
 ('남미', 0.06374073204204656)]

## PPMI + SVD 를 이용한 차원 축소

Singular Vector Decomposition (SVD) 를 이용하여 차원을 축소합니다.

In [12]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300)
x_pmisvd = svd.fit_transform(x_pmi)

SVD 로 차원 축소를 하면 noise 가 어느 정도 제거됩니다. Co-occurrence matrix 에서의 noise 는 문맥이 특이하거나 infrequent 한 단어입니다. 그 외에는 PPMI 를 적용했을 때와 유사어의 관계는 비슷합니다. 하지만 cosine 유사도의 값은 커집니다. PPMI 만 적용하면 sparse vector 이기 때문에 평균적인 유사도가 작습니다. 하지만 SVD 를 적용하면 평균적인 유사도 scale 이 커집니다.

In [13]:
most_similar_words('아이오아이', x_pmisvd)

[('블랙핑크', 0.8162061296682889),
 ('트와이스', 0.7910113638539613),
 ('블핑', 0.7679792737583678),
 ('빅뱅', 0.7438666999604329),
 ('여자친구', 0.7364958945398206),
 ('트와', 0.7304367399302416),
 ('엑소', 0.7201511797130391),
 ('ioi', 0.6927218648856788),
 ('아오아', 0.6839129116900139),
 ('음원', 0.6790602896134313)]

In [14]:
most_similar_words('아프리카', x_pmisvd)

[('유투브', 0.4844562611869139),
 ('tv', 0.46934051763480533),
 ('유튜브', 0.46592377270677743),
 ('티비', 0.44565890069529057),
 ('jtbc', 0.4327772253591923),
 ('TV', 0.4307487376379975),
 ('유럽', 0.42981356172467744),
 ('광고', 0.42824023679660894),
 ('채널', 0.4273553543926909),
 ('이슬람', 0.42663675730599715)]

## S, Sigma, V 를 이용하여 representation 과 mapper 만들기

In [15]:
from sklearn.utils import check_random_state
from sklearn.utils.extmath import randomized_svd

def train(X, n_components, n_iter=5, random_state=None):

    if (random_state == None) or isinstance(random_state, int):
        random_state = check_random_state(random_state)

    n_features = X.shape[1]

    if n_components >= n_features:
        raise ValueError("n_components must be < n_features;"
                         " got %d >= %d" % (n_components, n_features))

    U, Sigma, VT = randomized_svd(
        X, n_components,
        n_iter = n_iter,
        random_state = random_state)

    S_ = Sigma ** (0.5)
    representation = y = U * S_
    mapper = (VT.T * S_).T

    return representation, mapper

wv, mapper = train(x_pmi, n_components=300)

In [16]:
most_similar_words('아이오아이', wv)

[('블랙핑크', 0.7502299094221191),
 ('트와이스', 0.7484624801094757),
 ('블핑', 0.707471995322307),
 ('트와', 0.6661149639890701),
 ('ioi', 0.648183439924753),
 ('여자친구', 0.6457042531915209),
 ('엑소', 0.6432528443193453),
 ('빅뱅', 0.6415343264212899),
 ('유닛', 0.6111920479598711),
 ('음원', 0.6021137226519814)]

In [17]:
most_similar_words('아프리카', wv)

[('유튜브', 0.33624725785174747),
 ('bj', 0.33130966351946567),
 ('유투브', 0.3286154529671751),
 ('화질', 0.308646021163389),
 ('아시아', 0.30832490294227555),
 ('철구', 0.3048154094624257),
 ('중동', 0.30134489800460185),
 ('채널', 0.2938593187909968),
 ('들어오면', 0.286887943546003),
 ('광고', 0.2860743339772849)]